# Tutorial 1 main function

In [29]:
#!/usr/bin/env python3

# Standard python library

# Local library:
import IO.user_provided
import objective.setup_objective
import optimizer.gradient_free
import objective.compute_objective

# Third party library:

In [30]:
# In the shell, use the slurm job scheduler to request cores to run the job interactively

#[shell]$ srun  -n 2 -t 10:00:00 --x11=first --pty /bin/bash  -l

# the content of input file: 
#text_file = open("in_obj")
#file_content = text_file.read()
#print(file_content)
#text_file.close()

## units: 

real   

## Define objective function: 

force mW_300K_1bar_500 1 2 2 bf 5000 eng abs w 0.0 1.0 

## sampling command  # sampling input file names:  in.force, in.rdf, in.isobar
## sampling by default is LAMMPS 

module load intel && srun -n %d -N1 -c1 --mpi=pmi2 lmp_ml_water < in.%s

## output frequency: current best parameters/obj, restart simplex  

5 5 

##stillinger_weber 6.189  2.3925  1.80  23.15  1.20  -0.333333333333 7.049556277  0.6022245584  4.0  0.0 0.0  
stillinger_weber 6.589  2.0925  1.87  29.15  1.02  -0.303333333333 7.049556277  0.6822245584  3.2  0.4 0.0  

## fit (1) and fix (0) parameters: 

1 1 1 1 1 1 0 1 1 1 0   

## constraints (index lower upper)  ... ): 

10 0 5 

## set termination criterion: max number of iteration, tolerance for parameters,tolerance for objective 

25000 1e-14 1e-14

## create (Perturb) or use existing vertices (Restart): 

Nelder-Mead Perturb random 0.4  


In [32]:
#------------------------------------------------------------------------------
#                       Taking the input from user                             
#------------------------------------------------------------------------------
# global variables:
# main_logger: an object that defines the log file output
# ( you don't have to do anything with it )

# TOTAL_CORES: Number of cores assigned by slurm scheduler
# INPUT: a string of given input file name
# JOBID: a combination of Slurm job id and user-provided id



# For interactive job only:
main_logger,TOTAL_CORES,INPUT,JOBID,Ref,prep= (IO
                                         .user_provided
                                         .from_command_line(
                                            jobID="tutorial",
                                            total_cores=2,
                                            input_file="in_obj",
                                            ref_address="../force_matching_tutorial/ReferenceData",
                                            prep_address="../force_matching_tutorial/prepsystem")
                                         .finish_reading())


# For command line job only: 
"""
main_logger, TOTAL_CORES, INPUT, JOBID,Ref,prep = (IO
                                          .user_provided
                                          .from_command_line()
                                          .finish_reading())
""" 
print ("cores requested:", TOTAL_CORES,type(TOTAL_CORES))
print ("input file name:", INPUT,type(INPUT))
print ("Job id:", JOBID,type(JOBID))
print ("Reference data path:", Ref,type(Ref))
print ("prepsystem data path:", prep,type(prep))

cores requested: 2 <class 'int'>
input file name: in_obj <class 'str'>
Job id: tutorial <class 'str'>
Reference data path: ../force_matching_tutorial/ReferenceData <class 'str'>
prepsystem data path: ../force_matching_tutorial/prepsystem <class 'str'>


In [33]:
#------------------------------------------------------------------------------
#                           Set up the workflow                                
#------------------------------------------------------------------------------
# set up working folders
# Initialize sampling methods such LAMMPS
# Parse the arugment of objective functions from the input file

# Output: 
# ref_dict: a python dictionary containing reference data address
# predict_dict, a python dictionary containg predicted data address 
# argument_dict, a python dictionary containing arguments needed to run objective
# functions inclduing number of cores requested, buffersize etc ... 

ref_dict, predict_dict, argument_dict, LAMMPS, last_line = (objective
                                                            .setup_objective
                                                            .setup(
                                                                INPUT,
                                                                TOTAL_CORES,
                                                                JOBID,
                                                                overwrite=True,
                                                                Ref_folder=Ref,
                                                                prep_folder=prep)
                                                            .finish())


print ("referenced data dictionary:\n",ref_dict)

print ("predict data dictionary:\n",predict_dict)

print ("argument dictionary:",argument_dict)

referenced data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/force_matching_tutorial/ReferenceData/force/mW_300K_1bar_500',)}}
predict data dictionary:
 {'force': {'mW_300K_1bar_500': ('/project/palmer/Jingxiang/ours_optimization/tutorial/main_tutorial/tutorial/Predicted/force/mW_300K_1bar_500',)}}
argument dictionary: {'force': {'mW_300K_1bar_500': (1.0, 2, 2, 'bf 5000 eng abs w 0.0 1.0')}}


In [34]:
#------------------------------------------------------------------------------
#                           Initialize objective functions                     
#------------------------------------------------------------------------------

# Output: 
# instantiate the class object for computing specific objective fucntion  
# eval_objective has a method called "optimize", which takes the input
# of an array of optimized parameters and its type name (a string). 
# This eval_objective will be passed to optimizer 

eval_objective = (objective
                  .compute_objective
                  .prepare(
                    ref_dict,
                    predict_dict,
                    argument_dict,
                    LAMMPS))

In [35]:
#------------------------------------------------------------------------------
#                           start optimization                                 
#------------------------------------------------------------------------------

# Output: 
# optimize_fm: A python object initialized from the input file
# It contains a method called "run_optimization()"

""" 
optimize_fm = (optimizer
               .gradient_free
               .NelderMeadSimplex(
                   INPUT,
                   eval_objective,
                   skipped=last_line,
                   Output=JOBID+"/Output"))

# run optimization ...
optimize_fm.run_optimization()
"""


' \noptimize_fm = (optimizer\n               .gradient_free\n               .NelderMeadSimplex(\n                   INPUT,\n                   eval_objective,\n                   skipped=last_line,\n                   Output=JOBID+"/Output"))\n\n# run optimization ...\noptimize_fm.run_optimization()\n'